In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import cv2
import imutils
import numpy as np
import matplotlib.pyplot as plt
import time
from os import listdir

%matplotlib inline

In [0]:
def load_data(dir_list, image_size):
    """
    Read images, resize and normalize them. 
    Arguments:
        dir_list: list of strings representing file directories.
    Returns:
        X: A numpy array with shape = (#_examples, image_width, image_height, #_channels)
        y: A numpy array with shape = (#_examples, 1)
    """

    # load all images in a directory
    X = []
    y = []
    image_width, image_height = image_size
    
    for directory in dir_list:
        for filename in listdir(directory):
            # load the image
            
#             print(directory + '/' + filename)
            
            image = cv2.imread(directory + '/' + filename)
            
            # resize image
            image = cv2.resize(image, dsize=(image_width, image_height), interpolation=cv2.INTER_CUBIC)
            # normalize values
            # image = image / 255.
            # convert image to numpy array and append it to X
            X.append(image)
            # append a value of 1 to the target array if the image
            # is in the folder named 'yes', otherwise append 0.
            if directory[-3:] == 'yes':
                y.append([1])            
            else:
                y.append([0])
                
    X = np.array(X)
    y = np.array(y)
    print(y)
    
    # Shuffle the data
    X, y = shuffle(X, y)
    
    print(f'Number of examples is: {len(X)}')
    print(f'X shape is: {X.shape}')
    print(f'y shape is: {y.shape}')
    
    return X, y

In [0]:
augmented_path = '/content/drive/My Drive/Breast/augmented_data/'

# augmented data (yes and no) contains both the original and the new generated examples
augmented_yes = augmented_path + 'yes' 
augmented_no = augmented_path + 'no'


print(augmented_yes)
print(augmented_no)
# print(augmented_normal)

IMG_WIDTH, IMG_HEIGHT = (240, 240)

X, y = load_data([augmented_yes, augmented_no], (IMG_WIDTH, IMG_HEIGHT))

/content/drive/My Drive/Breast/augmented_data/yes
/content/drive/My Drive/Breast/augmented_data/no
[[1]
 [1]
 [1]
 ...
 [0]
 [0]
 [0]]
Number of examples is: 3865
X shape is: (3865, 240, 240, 3)
y shape is: (3865, 1)


In [0]:
# print(X)
print(y)

[[1]
 [1]
 [1]
 ...
 [1]
 [1]
 [1]]


In [0]:
def plot_sample_images(X, y, n=50):
    """
    Plots n sample images for both values of y (labels).
    Arguments:
        X: A numpy array with shape = (#_examples, image_width, image_height, #_channels)
        y: A numpy array with shape = (#_examples, 1)
    """
    
    for label in [0,1]:
        # grab the first n images with the corresponding y values equal to label
        images = X[np.argwhere(y == label)]
        n_images = images[:n]
        
        columns_n = 10
        rows_n = int(n/ columns_n)

        plt.figure(figsize=(20, 10))
        
        i = 1 # current plot        
        for image in n_images:
            plt.subplot(rows_n, columns_n, i)
            plt.imshow(image[0])
            
            # remove ticks
            plt.tick_params(axis='both', which='both', 
                            top=False, bottom=False, left=False, right=False,
                           labelbottom=False, labeltop=False, labelleft=False, labelright=False)
            
            i += 1
        
        label_to_str = lambda label: "Yes" if label == 1 else "No"
        plt.suptitle(f"Breast_cancer: {label_to_str(label)}")
        plt.show()

In [0]:
plot_sample_images(X, y)

In [0]:
def split_data(X, y, test_size=0.2):
       
    """
    Splits data into training, development and test sets.
    Arguments:
        X: A numpy array with shape = (#_examples, image_width, image_height, #_channels)
        y: A numpy array with shape = (#_examples, 1)
    Returns:
        X_train: A numpy array with shape = (#_train_examples, image_width, image_height, #_channels)
        y_train: A numpy array with shape = (#_train_examples, 1)
        X_val: A numpy array with shape = (#_val_examples, image_width, image_height, #_channels)
        y_val: A numpy array with shape = (#_val_examples, 1)
        X_test: A numpy array with shape = (#_test_examples, image_width, image_height, #_channels)
        y_test: A numpy array with shape = (#_test_examples, 1)
    """
    
    X_train, X_test_val, y_train, y_test_val = train_test_split(X, y, test_size=test_size)
    X_test, X_val, y_test, y_val = train_test_split(X_test_val, y_test_val, test_size=0.5)
    
    return X_train, y_train, X_val, y_val, X_test, y_test

In [0]:
X_train, y_train, X_val, y_val, X_test, y_test = split_data(X, y, test_size=0.3)

In [0]:
print ("number of training examples = " + str(X_train.shape[0]))
print ("number of development examples = " + str(X_val.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(y_train.shape))
print ("X_val (dev) shape: " + str(X_val.shape))
print ("Y_val (dev) shape: " + str(y_val.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(y_test.shape))

number of training examples = 2705
number of development examples = 580
number of test examples = 580
X_train shape: (2705, 240, 240, 3)
Y_train shape: (2705, 1)
X_val (dev) shape: (580, 240, 240, 3)
Y_val (dev) shape: (580, 1)
X_test shape: (580, 240, 240, 3)
Y_test shape: (580, 1)


In [0]:
# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return f"{h}:{m}:{round(s,1)}"

In [0]:
def compute_f1_score(y_true, prob):
    # convert the vector of probabilities to a target vector
    y_pred = np.where(prob > 0.5, 1, 0)
    
    score = f1_score(y_true, y_pred)
    
    return score

In [0]:
def build_model(input_shape):
    """
    Arugments:
        input_shape: A tuple representing the shape of the input of the model. shape=(image_width, image_height, #_channels)
    Returns:
        model: A Model object.
    """
    # Define the input placeholder as a tensor with shape input_shape. 
    X_input = Input(input_shape) # shape=(?, 240, 240, 3)
    
    # Zero-Padding: pads the border of X_input with zeroes
    X = ZeroPadding2D((2, 2))(X_input) # shape=(?, 244, 244, 3)
    
    # CONV -> BN -> RELU Block applied to X
    X = Conv2D(32, (7, 7), strides = (1, 1), name = 'conv0')(X)
    X = BatchNormalization(axis = 3, name = 'bn0')(X)
    X = Activation('relu')(X) # shape=(?, 238, 238, 32)
    
    # MAXPOOL
    X = MaxPooling2D((4, 4), name='max_pool0')(X) # shape=(?, 59, 59, 32) 
    
    # MAXPOOL
    X = MaxPooling2D((4, 4), name='max_pool1')(X) # shape=(?, 14, 14, 32)
    
    # FLATTEN X 
    X = Flatten()(X) # shape=(?, 6272)
    # FULLYCONNECTED
    X = Dense(1, activation='sigmoid', name='fc')(X) # shape=(?, 1)
    
    # Create model. This creates your Keras model instance, you'll use this instance to train/test the model.
    model = Model(inputs = X_input, outputs = X, name='Breast_cancer_DetectionModel')
    
    return model

In [0]:
IMG_SHAPE = (IMG_WIDTH, IMG_HEIGHT, 3)

In [0]:
model = build_model(IMG_SHAPE)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
model.summary()

Model: "Breast_cancer_DetectionModel"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 240, 240, 3)]     0         
_________________________________________________________________
zero_padding2d (ZeroPadding2 (None, 244, 244, 3)       0         
_________________________________________________________________
conv0 (Conv2D)               (None, 238, 238, 32)      4736      
_________________________________________________________________
bn0 (BatchNormalization)     (None, 238, 238, 32)      128       
_________________________________________________________________
activation (Activation)      (None, 238, 238, 32)      0         
_________________________________________________________________
max_pool0 (MaxPooling2D)     (None, 59, 59, 32)        0         
_________________________________________________________________
max_pool1 (MaxPooling2D)     (None, 14

In [0]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
# tensorboard
log_file_name = f'breast_cancer_detection_cnn_{int(time.time())}'
tensorboard = TensorBoard(log_dir=f'/content/drive/My Drive/Breast/logs/{log_file_name}')

In [0]:
# checkpoint
# unique file name that will include the epoch and the validation (development) accuracy
filepath="cnn-parameters-improvement-{epoch:02d}-{val_acc:.2f}"
# save the model with the best validation (development) accuracy till now
checkpoint = ModelCheckpoint("/content/drive/My Drive/Breast/models/{}.model".format(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max'))

In [0]:
start_time = time.time()

model.fit(x=X_train, y=y_train, batch_size=32, epochs=50, validation_data=(X_val, y_val), callbacks=[tensorboard, checkpoint])

end_time = time.time()
execution_time = (end_time - start_time)
print(f"Elapsed time: {hms_string(execution_time)}")

Train on 2705 samples, validate on 580 samples
Epoch 1/50
2705/2705 [==============================] - 10s 4ms/sample - loss: 0.7135 - acc: 0.6488 - val_loss: 0.6392 - val_acc: 0.6724
Epoch 2/50
2705/2705 [==============================] - 3s 1ms/sample - loss: 0.6075 - acc: 0.6891 - val_loss: 0.5972 - val_acc: 0.7103
Epoch 3/50
2705/2705 [==============================] - 3s 1ms/sample - loss: 0.5594 - acc: 0.7275 - val_loss: 0.6241 - val_acc: 0.6552
Epoch 4/50
2705/2705 [==============================] - 3s 1ms/sample - loss: 0.5607 - acc: 0.7198 - val_loss: 0.8042 - val_acc: 0.6000
Epoch 5/50
2705/2705 [==============================] - 3s 1ms/sample - loss: 0.6009 - acc: 0.7031 - val_loss: 0.5822 - val_acc: 0.7190
Epoch 6/50
2705/2705 [==============================] - 3s 1ms/sample - loss: 0.6033 - acc: 0.6972 - val_loss: 0.5812 - val_acc: 0.7293
Epoch 7/50
2705/2705 [==============================] - 3s 1ms/sample - loss: 0.5528 - acc: 0.7327 - val_loss: 0.6035 - val_acc: 0.6862


In [0]:
start_time = time.time()

model.fit(x=X_train, y=y_train, batch_size=32, epochs=20, validation_data=(X_val, y_val), callbacks=[tensorboard, checkpoint])

end_time = time.time()
execution_time = (end_time - start_time)
print(f"Elapsed time: {hms_string(execution_time)}")

Train on 2705 samples, validate on 580 samples
Epoch 1/20
2705/2705 [==============================] - 3s 1ms/sample - loss: 0.1982 - acc: 0.9227 - val_loss: 0.5709 - val_acc: 0.7690
Epoch 2/20
2705/2705 [==============================] - 3s 1ms/sample - loss: 0.1870 - acc: 0.9283 - val_loss: 0.6177 - val_acc: 0.7655
Epoch 3/20
2705/2705 [==============================] - 3s 1ms/sample - loss: 0.1740 - acc: 0.9416 - val_loss: 0.6338 - val_acc: 0.7483
Epoch 4/20
2705/2705 [==============================] - 3s 1ms/sample - loss: 0.1753 - acc: 0.9372 - val_loss: 0.8569 - val_acc: 0.6966
Epoch 5/20
2705/2705 [==============================] - 3s 1ms/sample - loss: 0.1764 - acc: 0.9327 - val_loss: 0.6961 - val_acc: 0.7345
Epoch 6/20
2705/2705 [==============================] - 3s 1ms/sample - loss: 0.1495 - acc: 0.9519 - val_loss: 0.7557 - val_acc: 0.7121
Epoch 7/20
2705/2705 [==============================] - 3s 1ms/sample - loss: 0.1721 - acc: 0.9360 - val_loss: 0.6484 - val_acc: 0.7466
E

In [0]:
start_time = time.time()

model.fit(x=X_train, y=y_train, batch_size=32, epochs=20, validation_data=(X_val, y_val), callbacks=[tensorboard, checkpoint])

end_time = time.time()
execution_time = (end_time - start_time)
print(f"Elapsed time: {hms_string(execution_time)}")

Train on 2705 samples, validate on 580 samples
Epoch 1/20
2705/2705 [==============================] - 4s 1ms/sample - loss: 0.1187 - acc: 0.9652 - val_loss: 0.7151 - val_acc: 0.7707
Epoch 2/20
2705/2705 [==============================] - 4s 1ms/sample - loss: 0.1089 - acc: 0.9652 - val_loss: 0.7379 - val_acc: 0.7707
Epoch 3/20
2705/2705 [==============================] - 3s 1ms/sample - loss: 0.1064 - acc: 0.9667 - val_loss: 0.7494 - val_acc: 0.7603
Epoch 4/20
2705/2705 [==============================] - 3s 1ms/sample - loss: 0.0882 - acc: 0.9774 - val_loss: 0.7807 - val_acc: 0.7552
Epoch 5/20
2705/2705 [==============================] - 3s 1ms/sample - loss: 0.1064 - acc: 0.9612 - val_loss: 0.7663 - val_acc: 0.7552
Epoch 6/20
2705/2705 [==============================] - 3s 1ms/sample - loss: 0.0795 - acc: 0.9841 - val_loss: 0.9073 - val_acc: 0.7397
Epoch 7/20
2705/2705 [==============================] - 3s 1ms/sample - loss: 0.0976 - acc: 0.9652 - val_loss: 0.9007 - val_acc: 0.7121
E

In [0]:
start_time = time.time()

model.fit(x=X_train, y=y_train, batch_size=32, epochs=30, validation_data=(X_val, y_val), callbacks=[tensorboard, checkpoint])

end_time = time.time()
execution_time = (end_time - start_time)
print(f"Elapsed time: {hms_string(execution_time)}")

Train on 2705 samples, validate on 580 samples
Epoch 1/30
2705/2705 [==============================] - 3s 1ms/sample - loss: 0.0537 - acc: 0.9904 - val_loss: 0.8922 - val_acc: 0.7552
Epoch 2/30
2705/2705 [==============================] - 3s 1ms/sample - loss: 0.0432 - acc: 0.9959 - val_loss: 0.9018 - val_acc: 0.7621
Epoch 3/30
2705/2705 [==============================] - 3s 1ms/sample - loss: 0.0417 - acc: 0.9948 - val_loss: 0.8991 - val_acc: 0.7448
Epoch 4/30
2705/2705 [==============================] - 4s 1ms/sample - loss: 0.0480 - acc: 0.9915 - val_loss: 0.8762 - val_acc: 0.7569
Epoch 5/30
2705/2705 [==============================] - 4s 1ms/sample - loss: 0.0776 - acc: 0.9726 - val_loss: 1.0644 - val_acc: 0.7172
Epoch 6/30
2705/2705 [==============================] - 3s 1ms/sample - loss: 0.0599 - acc: 0.9863 - val_loss: 0.9054 - val_acc: 0.7638
Epoch 7/30
2705/2705 [==============================] - 3s 1ms/sample - loss: 0.0590 - acc: 0.9845 - val_loss: 1.0138 - val_acc: 0.7414
E

In [0]:
X_test.shape

(580, 240, 240, 3)

In [0]:
def load_data1(file_path,image_size):
    
    X2 = []
    
    image_width, image_height = image_size
    print(file_path)
    image = cv2.imread(file_path)
    # crop the brain and ignore the unnecessary rest part of the image
    # image = crop_brain_contour(image, plot=False)
    # resize image
    image = cv2.resize(image, dsize=(image_width, image_height), interpolation=cv2.INTER_CUBIC)
    # normalize values
    # image = image / 255.
    # convert image to numpy array and append it to X
    X2.append(image)
    
    X2 = np.array(X2)
    
    print(f'Number of examples is: {len(X2)}')
    print(f'X shape is: {X2.shape}')
    
    return X2

In [0]:
file_path="/content/drive/My Drive/Breast/malignant/malignant (12).png"
img=load_data1(file_path,(IMG_WIDTH, IMG_HEIGHT))

/content/drive/My Drive/Breast/malignant/malignant (12).png
Number of examples is: 1
X shape is: (1, 240, 240, 3)


In [0]:
best_model = load_model(filepath='/content/drive/My Drive/Breast/models/cnn-parameters-improvement-50-0.76.model')

ss=best_model.predict(img)

print(ss)

if ss[0][0]>0.2:
    output="Affected"
else:
    output="Not Affected"
        
print(output)

[[0.7918848]]
Affected
